# Merge Predictions and OOF for Giba

In [2]:
import pandas as pd
import os

In [4]:
types = ['1JHC', '2JHH', '1JHN', '2JHN', '2JHC','3JHH','3JHC', '3JHN']
oof = pd.read_csv('../input/train.csv')
sub = pd.read_csv('../input/test.csv')
sub_final = pd.DataFrame()
oof_final = pd.DataFrame()
sub = sub.drop(['molecule_name','atom_index_0','atom_index_1'], axis=1)
oof = oof.drop(['molecule_name','atom_index_0','atom_index_1'], axis=1)

for t in types:
    subt = sub.loc[sub['type'] == t].copy()
    ooft = oof.loc[oof['type'] == t].copy()
    print(f'====== Running for type {t} ========')
    for file in os.listdir(f'../type_results/{t}/'):
        if '3folds' in file:
    #        print(file)
            if 'oof' in file:
                df = pd.read_parquet(f'../type_results/{t}/{file}')
                ooft = ooft.merge(df[['id','oof_preds']] \
                              .rename(columns={'oof_preds': file[:4]}),
                          on='id')
            if 'sub' in file:
                df = pd.read_parquet(f'../type_results/{t}/{file}')
                subt = subt.merge(df[['id','scalar_coupling_constant']] \
                              .rename(columns={'scalar_coupling_constant': file[:4]}),
                          on='id')
                print(file)
    #             df = pd.read_parquet(f'../type_results/{t}/{file}')
    #             sub = pd.concat([sub, df])
    sub_final = pd.concat([subt, sub_final], sort=False)
    oof_final = pd.concat([ooft, oof_final], sort=False)

====== Running for type 1JHC ========
M048_0718_1745_1JHC_sub_0.4379MAE_-0.8258LMAE_catboost_3folds_500000iter_0.1lr.parquet
M049_0721_0839_1JHC_sub_0.4534MAE_-0.791LMAE_xgboost_3folds_500000iter_0.05lr.parquet
M047_0714_2226_1JHC_submission_0.4504MAE_-0.7977LMAE_catboost_3folds_500000iter_0.1lr.parquet
====== Running for type 2JHH ========
M047_0714_2226_2JHH_submission_0.1098MAE_-2.209LMAE_catboost_3folds_500000iter_0.1lr.parquet
M049_0719_1817_2JHH_sub_0.1159MAE_-2.155LMAE_xgboost_3folds_100000iter_0.1lr.parquet
M050_0721_2252_2JHH_sub_0.1190MAE_-2.129LMAE_xgboost_3folds_500000iter_0.005lr.parquet
M048_0717_2239_2JHH_sub_0.1020MAE_-2.282LMAE_catboost_3folds_500000iter_0.1lr.parquet
====== Running for type 1JHN ========
M049_0719_2005_1JHN_sub_0.2565MAE_-1.361LMAE_xgboost_3folds_100000iter_0.005lr.parquet
M047_0714_2226_1JHN_submission_0.3276MAE_-1.116LMAE_catboost_3folds_500000iter_0.1lr.parquet
M048_0717_2239_1JHN_sub_0.3367MAE_-1.089LMAE_catboost_3folds_500000iter_0.1lr.parquet
M0

In [5]:
oof_final.shape

(4658147, 8)

In [6]:
sub_final.shape

(2505542, 7)

In [7]:
oof.shape

(4658147, 3)

In [8]:
sub.shape

(2505542, 2)

In [9]:
oof_final.loc[oof_final['type'] == '3JHN'].head()

,id,type,scalar_coupling_constant,M047,M049,M052,M050,M048
0,73,3JHN,0.880802,1.217714,1.085314,0.907007,0.871911,1.006925
1,78,3JHN,0.880957,1.199598,1.082283,0.908139,0.873156,0.948375
2,82,3JHN,0.880871,1.196354,1.063929,0.863326,0.876223,1.054770
3,213,3JHN,-0.052074,0.028468,0.154375,0.037855,-0.135423,-0.046810
4,218,3JHN,1.611320,2.005109,1.473568,1.819162,2.285958,1.669883


In [10]:
sub_final.loc[sub_final['type'] == '3JHN'].head()

,id,type,M049,M048,M047,M050,M052
0,4658205,3JHN,0.160267,-0.176915,-0.237696,-0.060738,-0.268147
1,4658209,3JHN,1.995038,2.426134,2.484086,2.678819,6.613277
2,4658210,3JHN,0.146129,-0.199509,-0.256149,-0.060738,-0.291760
3,4658214,3JHN,2.005161,2.425806,2.537887,2.678819,6.625949
4,4658555,3JHN,-0.010291,-0.242351,-0.275497,-0.317335,-0.785255


In [98]:
oof_final.columns

Index(['id', 'type', 'scalar_coupling_constant', 'M047', 'M049', 'M052',
       'M050', 'M048'],
      dtype='object')

In [100]:
oof_final = oof_final[['id', 'type', 'scalar_coupling_constant', 'M047', 'M048', 'M049', 'M050', 'M052']]

In [101]:
sub_final.columns

Index(['id', 'type', 'M049', 'M048', 'M047', 'M050', 'M052'], dtype='object')

In [103]:
sub_final = sub_final[['id', 'type', 'M047', 'M048', 'M049', 'M050', 'M052']]

In [ ]:
sub_final.to_csv('../submissions/MERGE015-sub-ROB_BestModels_Jul24.csv', index=False)
oof_final.to_csv('../oof/MERGE015-oof-ROB_BestModels_Jul24.csv', index=False)